In [1]:
import pandas as pd
import folium
import math
import branca.colormap as cm
import numpy as np

# --- 1. Загрузка и подготовка данных ---
try:
    # Загрузка данных о зонах
    zones_df = pd.read_csv('well_clusters_detailed_analysis.csv')
    # Загрузка категорий зон
    categories_df = pd.read_csv('zone_categories.csv')

    # Объединение данных по zone_id
    map_data = pd.merge(zones_df, categories_df, on='zone_id', how='left')

    # Заполнение отсутствующих категорий (если есть)
    map_data['category'].fillna('Unknown', inplace=True)

    # Проверка наличия необходимых колонок
    required_cols = ['zone_id', 'zone_center_latitude', 'zone_center_longitude',
                     'zone_area_km2', 'zone_num_wells', 'category',
                     'zone_average_gradient', 'zone_average_bht']
    if not all(col in map_data.columns for col in required_cols):
        raise ValueError(f"Отсутствуют необходимые колонки. Требуются: {required_cols}")

    # Удаление строк с отсутствующими координатами или площадью
    map_data.dropna(subset=['zone_center_latitude', 'zone_center_longitude', 'zone_area_km2'], inplace=True)

    if map_data.empty:
        print("Нет данных для отображения после очистки.")
        exit()

except FileNotFoundError as e:
    print(f"Ошибка: Файл не найден - {e}. Убедитесь, что файлы CSV находятся в той же директории.")
    exit()
except ValueError as e:
    print(f"Ошибка данных: {e}")
    exit()
except Exception as e:
    print(f"Произошла непредвиденная ошибка при загрузке данных: {e}")
    exit()

# --- 2. Настройка карты ---

# Определение центра карты (среднее значение координат)
center_lat = map_data['zone_center_latitude'].mean()
center_lon = map_data['zone_center_longitude'].mean()

# Создание объекта карты
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='CartoDB positron')

# --- 3. Определение цветов и насыщенности ---

# Словарь цветов для категорий
color_map = {
    'Electricity generation': '#0000FF', # Синий
    'Heat generation': '#FF0000',       # Красный
    'Mixed': '#008000',                 # Зеленый
    'Unknown': '#808080'                # Серый для неизвестных
}

# Нормализация количества скважин для определения насыщенности (opacity)
# Используем логарифмическое масштабирование для лучшего визуального разделения,
# если разброс значений велик. Добавляем 1, чтобы избежать log(0).
min_wells = map_data['zone_num_wells'].min()
max_wells = map_data['zone_num_wells'].max()

# Проверка, что max_wells > min_wells, чтобы избежать деления на ноль
if max_wells > min_wells:
    # Нормализация в диапазон [0.3, 1.0] для непрозрачности (alpha)
     map_data['opacity'] = 0.3 + 0.7 * (map_data['zone_num_wells'] - min_wells) / (max_wells - min_wells)
     # Для логарифмического масштабирования (если нужно):
     # map_data['log_wells'] = np.log1p(map_data['zone_num_wells'])
     # min_log_wells = map_data['log_wells'].min()
     # max_log_wells = map_data['log_wells'].max()
     # if max_log_wells > min_log_wells:
     #     map_data['opacity'] = 0.3 + 0.7 * (map_data['log_wells'] - min_log_wells) / (max_log_wells - min_log_wells)
     # else:
     #     map_data['opacity'] = 0.65 # Среднее значение, если все значения одинаковы
else:
    map_data['opacity'] = 0.65 # Используем среднее значение, если все зоны имеют одинаковое кол-во скважин

# --- 4. Добавление зон на карту ---

for idx, row in map_data.iterrows():
    # Получение базового цвета
    base_color_hex = color_map.get(row['category'], '#808080') # Серый по умолчанию

    # Получение непрозрачности
    opacity = row['opacity']

    # Расчет радиуса в метрах (Area = pi * r^2 => r = sqrt(Area / pi))
    radius_meters = math.sqrt(row['zone_area_km2'] / math.pi) * 1000

    # Создание всплывающего окна с информацией
    popup_html = f"""
    <b>Зона ID:</b> {row['zone_id']}<br>
    <b>Категория:</b> {row['category']}<br>
    <b>Кол-во скважин:</b> {row['zone_num_wells']}<br>
    <b>Площадь:</b> {row['zone_area_km2']:.2f} км²<br>
    <b>Средний градиент:</b> {row['zone_average_gradient']:.2f}<br>
    <b>Средняя температура (BHT):</b> {row['zone_average_bht']:.2f}
    """
    popup = folium.Popup(popup_html, max_width=300)

    # Добавление круга на карту
    folium.Circle(
        location=[row['zone_center_latitude'], row['zone_center_longitude']],
        radius=radius_meters,
        popup=popup,
        tooltip=f"Зона {row['zone_id']} ({row['category']})",
        color=base_color_hex, # Цвет контура
        fill=True,
        fill_color=base_color_hex, # Цвет заливки
        fill_opacity=opacity # Насыщенность/непрозрачность
    ).add_to(m)

# --- 5. Добавление легенды ---

# Легенда для категорий
legend_html_categories = '''
     <div style="position: fixed;
     bottom: 50px; right: 10px; width: 180px; height: 100px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:rgba(255, 255, 255, 0.8); /* Полупрозрачный фон */
     border-radius: 5px; /* Скругленные углы */
     padding: 5px; /* Отступы */
     ">
     <b>Категории зон</b><br>
     <i class="fa fa-circle" style="color:#0000FF"></i>&nbsp; Electricity generation<br>
     <i class="fa fa-circle" style="color:#FF0000"></i>&nbsp; Heat generation<br>
     <i class="fa fa-circle" style="color:#008000"></i>&nbsp; Mixed<br>
     <i class="fa fa-circle" style="color:#808080"></i>&nbsp; Unknown<br>
      </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html_categories))

# Легенда для насыщенности (шкала)
# Создаем цветовую шкалу от минимального до максимального числа скважин
# Используем линейную шкалу branca для градиента насыщенности (используем оттенки серого для примера)
# Для реальной насыщенности цвета, это сложнее сделать в HTML/CSS легенде без JS.
# Эта легенда показывает диапазон количества скважин.
min_wells_display = int(min_wells)
max_wells_display = int(max_wells)

legend_html_saturation = f'''
     <div style="position: fixed;
     bottom: 160px; right: 10px; width: 180px; height: 80px;
     border:2px solid grey; z-index:9998; font-size:14px;
     background-color:rgba(255, 255, 255, 0.8);
     border-radius: 5px;
     padding: 5px;
     ">
     <b>Насыщенность (Кол-во скважин)</b><br>
     <div style="display: flex; align-items: center;">
       <span style="opacity: 0.3; font-size: 20px; margin-right: 5px;">■</span> <span style="flex-grow: 1; text-align: center;">{min_wells_display}</span>
     </div>
     <div style="background: linear-gradient(to right, rgba(128,128,128,0.3), rgba(128,128,128,1)); height: 10px; margin-top: 2px; margin-bottom: 2px;"></div> <div style="display: flex; align-items: center;">
       <span style="opacity: 1.0; font-size: 20px; margin-right: 5px;">■</span> <span style="flex-grow: 1; text-align: center;">{max_wells_display}</span>
     </div>
     </div>
     '''
# Добавляем Font Awesome CSS для иконок легенды категорий (если еще не подключен)
# Обычно folium подключает его, но на всякий случай:
# fa_css = '<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">'
# m.get_root().html.add_child(folium.Element(fa_css))

m.get_root().html.add_child(folium.Element(legend_html_saturation))


# --- 6. Сохранение карты ---
output_filename = 'map_zones.html'
m.save(output_filename)

print(f"Карта успешно сохранена в файл: {output_filename}")
print("Примечание: Для сохранения в PNG откройте HTML файл в браузере (например, Chrome) и используйте функцию 'Сохранить как изображение' или 'Сделать скриншот'.")
print("Альтернативно, можно использовать библиотеки Python, такие как Selenium и Pillow, для автоматического сохранения PNG, но это требует установки веб-драйвера.")



C:\Users\yusif\AppData\Local\Temp\ipykernel_22092\68219488.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  map_data['category'].fillna('Unknown', inplace=True)


Карта успешно сохранена в файл: map_zones.html
Примечание: Для сохранения в PNG откройте HTML файл в браузере (например, Chrome) и используйте функцию 'Сохранить как изображение' или 'Сделать скриншот'.
Альтернативно, можно использовать библиотеки Python, такие как Selenium и Pillow, для автоматического сохранения PNG, но это требует установки веб-драйвера.
